In [ ]:
# Things move fast
# Old:
with tokenizer.as_target_tokenizer():
  targets = tokenizer(fr_sentence)

# New:
targets = tokenizer(text_targets=es_sentence)

In [1]:
# sentencepiece is req. for the tokenizer to work
!pip install transformers datasets sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [3]:
#
from datasets import load_dataset
# possible language pairs: https://opus.nlpl.eu/KDE4.php
data = load_dataset("kde4", lang1="en", lang2="sv")
data

Generating train split: 0 examples [00:00, ? examples/s]

Dataset kde4 downloaded and prepared to /root/.cache/huggingface/datasets/kde4/en-sv-lang1=en,lang2=sv/0.0.0/243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 232485
    })
})

In [4]:
# make the dataset managable
small = data["train"].shuffle(seed=42).select(range(1_000))

In [6]:
# split the dataset into train and test
split = small.train_test_split(seed=42)

In [7]:
# print out one sample from the dataset
split["train"][0]

{'id': '95084', 'translation': {'en': 'Download', 'sv': 'Ladda ner'}}

In [8]:
# load up the tokenizer object
from transformers import AutoTokenizer

checkpoint = "Helsinki-NLP/opus-mt-en-sv"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [9]:
# print out anexample sentence pair
en = split['train'][5]['translation']['en']
sv = split['train'][5]['translation']['sv']

en, sv

('with this Polygonal', 'med den här polygonformen')

In [10]:
# tokenize the input sentence
inputs = tokenizer(en)
inputs

{'input_ids': [44, 56, 12284, 5844, 314, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [11]:
# tokenize the target sentence
with tokenizer.as_target_tokenizer():
  targets = tokenizer(sv)
targets

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [30, 35, 93, 4721, 5844, 30212, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# look at the tokens for the target
tokenizer.convert_ids_to_tokens(targets['input_ids'])

In [ ]:
# wrong language
bad_targets = tokenizer(sv)
tokenizer.convert_ids_to_tokens(bad_targets['input_ids'])

In [ ]:
# 
import matplotlib.pyplot as plt

train = split['train']['translation']
input_lens = [len(tr['en']) fot tr in train]

plt.hist(input_lens, bins=50);

In [ ]:
# 